# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1+cu117
CUDA version: 11.7
cuDNN version: 8500
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.32.1
Datasets version: 2.14.4


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Sun Dec  3 08:55:47 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 24%   44C    P2               62W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |   0% |  8% |
|  1 |   0% | 46% |
|  2 |  99% | 73% |
|  3 |  99% | 73% |
|  4 |   0% | 12% |
|  5 | 100% | 73% |
|  6 | 100% | 73% |
|  7 |   0% |  0% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |   0% |  8% |
|  1 |   0% | 46% |
|  2 | 100% | 73% |
|  3 | 100% | 73% |
|  4 |   0% | 12% |
|  5 |  98% | 73% |
|  6 |  99% | 73% |
|  7 |   1% |  1% |


In [8]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 3 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [9]:
dataset = load_dataset("kristmh/high_vs_med_and_low_priority")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/42484 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/339866 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/42483 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 42484
    })
    train: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 339866
    })
    validate: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__'],
        num_rows: 42483
    })
})

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

## Tokenization

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [14]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [15]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map:   0%|          | 0/42484 [00:00<?, ? examples/s]

Map:   0%|          | 0/339866 [00:00<?, ? examples/s]

Map:   0%|          | 0/42483 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 42484
    })
    train: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 339866
    })
    validate: Dataset({
        features: ['text_clean', 'labels', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 42483
    })
})

In [16]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean","class", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 339866
})
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 42483
})


Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 42484
})

## Training a classifier

In [17]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [19]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.581800,0.611503,0.667867,[0.69706728 0.63243722],[0.64407856 0.70256974],[0.75955645 0.57503553]
2,0.578700,0.601308,0.677330,[0.69007461 0.66349175],[0.66767292 0.68857638],[0.71403172 0.64017054]
3,0.524100,0.607112,0.680978,[0.68654162 0.67521388],[0.67883278 0.68325331],[0.69442755 0.66736144]


TrainOutput(global_step=63726, training_loss=0.5768061621326872, metrics={'train_runtime': 27708.0494, 'train_samples_per_second': 36.798, 'train_steps_per_second': 2.3, 'total_flos': 1.3506349473486029e+17, 'train_loss': 0.5768061621326872, 'epoch': 3.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [20]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [21]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.6809782736624062

eval_f1 = [0.68654162 0.67521388]

eval_loss = 0.6071115136146545

eval_precision = [0.67883278 0.68325331]

eval_recall = [0.69442755 0.66736144]

eval_runtime = 409.2773

eval_samples_per_second = 103.8

eval_steps_per_second = 3.245



In [22]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [23]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 3.0

eval_accuracy = 0.6803737877789285

eval_f1 = [0.68387848 0.67679052]

eval_loss = 0.604187548160553

eval_precision = [0.67842956 0.68239416]

eval_recall = [0.68941563 0.67127815]

eval_runtime = 419.1948

eval_samples_per_second = 101.347

eval_steps_per_second = 3.168



In [24]:
trainer.save_model(model_dir + "_local") 

In [25]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [26]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [27]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.5984984636306763}]

In [28]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.8423776626586914}]

In [29]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.8455770611763}]

## Important to delete large objects to free memory 
del train_dataset

In [30]:
del validation_dataset 

In [31]:
del model

In [32]:
# Free cache
torch.cuda.empty_cache()

In [33]:
!nvidia-smi

Sun Dec  3 17:02:17 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 24%   44C    P2               63W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--